In [2]:
pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.2 MB/s eta 0:00:00


In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

class MovieSpider(scrapy.Spider):
    name = "movie_spider"
    start_urls = ["https://www.rottentomatoes.com/browse/movies_at_home/sort:popular"]
    custom_settings = {
        'FEED_URI': 'movies.json',
        'FEED_FORMAT': 'json',
    }
    count = 0
    max_count = 500000

    def parse(self, response):
        movie_links = response.css("a.js-tile-link::attr(href)").getall()
        for link in movie_links:
            if self.count >= self.max_count:
                break
            url = response.urljoin(link)
            yield scrapy.Request(url, callback=self.parse_movie_details)

    def parse_movie_details(self, response):
        if self.count >= self.max_count:
            return
        self.count += 1
        yield {
            'url': response.url,
            'title': response.css("h1.mop-ratings-wrap__title::text").get(),
            'description': response.css("div.movie_synopsis::text").get().strip(),
            'genre': response.css("div.meta-value.genre a::text").getall(),
            'duration': response.css("time::text").get(),
            'director': response.css("a[slot=director]::text").getall(),
            'actors': response.css("a[slot=cast]::text").getall(),
            'user_score': response.css("span.mop-ratings-wrap__percentage::text").getall()[0],
            'critic_score': response.css("span.mop-ratings-wrap__percentage::text").getall()[1],
        }

class CriticReviewSpider(scrapy.Spider):
    name = "critic_review_spider"
    start_urls = ["https://www.rottentomatoes.com/critics/latest_reviews"]
    custom_settings = {
        'FEED_URI': 'critic_reviews.json',
        'FEED_FORMAT': 'json',
    }
    count = 0
    max_count = 500000

    def parse(self, response):
        review_links = response.css("a[href*='/critic-reviews/']::attr(href)").getall()
        for link in review_links:
            if self.count >= self.max_count:
                break
            url = response.urljoin(link)
            yield scrapy.Request(url, callback=self.parse_review_details)

    def parse_review_details(self, response):
        if self.count >= self.max_count:
            return
        self.count += 1
        yield {
            'url': response.url,
            'critic_name': response.css("h1.critic-name::text").get(),
            'date': response.css("time::text").get(),
            'rating': response.css("div.critic-score span::text").get(),
            'review_content': response.css("div.the_review::text").get(),
        }

if __name__ == "__main__":
    process = CrawlerProcess(get_project_settings())
    process.crawl(MovieSpider)
    process.crawl(CriticReviewSpider)
    process.start()


ModuleNotFoundError: No module named 'scrapy'